We'll use https://cassiopeia.readthedocs.io/en/latest/ to download data.

In [1]:
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [2]:
from cassiopeia import Champion, Champions, Summoner

import cassiopeia as cass

cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("NA")

In [5]:
all_champions = Champions(region="NA")
teemo = all_champions["Teemo"]
a_teemo_game = Summoner(account=50330641, region="NA").match_history[teemo]

Making call: https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/50330641?beginIndex=0&endIndex=100
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json


In [6]:
import random

import cassiopeia as cass

cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

summoner = cass.get_summoner(name="dskjbfksaf")
print("{name} is a level {level} summoner on the {region} server.".format(name=summoner.name,
                                                                          level=summoner.level,
                                                                          region=summoner.region))

champions = cass.get_champions()
random_champion = random.choice(champions)
print("He enjoys playing champions such as {name}.".format(name=random_champion.name))

challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives)
best_na = challenger_league[0].summoner
print("He's not as good as {name} at League, but probably a better python programmer!".format(name=best_na.name))

Making call: https://kr.api.riotgames.com/lol/summoner/v3/summoners/by-name/dskjbfksaf
dskjbfksaf is a level 59 summoner on the Region.korea server.
Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/championFull.json
He enjoys playing champions such as 파이크.
Making call: https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5
He's not as good as 오리지널찰떡쿠키 at League, but probably a better python programmer!


In [7]:
import cassiopeia as cass

summoner = cass.get_summoner(name="브라질리언했음")
match_hist = cass.get_match_history(summoner)

Making call: https://kr.api.riotgames.com/lol/summoner/v3/summoners/by-name/\xeb\xb8\x8c\xeb\x9d\xbc\xec\xa7\x88\xeb\xa6\xac\xec\x96\xb8\xed\x96\x88\xec\x9d\x8c


In [8]:
for i in range(10) : 
    print(match_hist[0].participants[i].summoner.name)

Making call: https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/5500568?beginIndex=0&endIndex=100
Making call: https://kr.api.riotgames.com/lol/summoner/v3/summoners/by-account/5500568
브라질리언했음
Making call: https://kr.api.riotgames.com/lol/match/v3/matches/3449425180
소중한한명
벤젠덴겐
머스켓
피들장인 최빨대
탐스런콧구멍
까치래도차케
horosha
시로조아
Ezreal Lover


In [9]:
match_hist[0].id

3449425180

In [10]:
import cassiopeia as cass

match_hist[0].map.name

Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/map.json


'소환사의 협곡'

In [11]:
match_hist = cass.get_match_history(summoner)

In [12]:
for i in range(5) : 
    if match_hist[0].teams[0].win : 
        print('blue team : ', match_hist[0].teams[0].participants[i].summoner.name, 'win')
    else :
        print('blue team : ', match_hist[0].teams[0].participants[i].summoner.name, 'loose')
for i in range(5) :
    if match_hist[0].teams[1].win : 
        print('red team : ', match_hist[0].teams[1].participants[i].summoner.name, 'win')
    else :
        print('red team : ', match_hist[0].teams[1].participants[i].summoner.name, 'loose')

Making call: https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/5500568?beginIndex=0&endIndex=100
blue team :  소중한한명 win
blue team :  벤젠덴겐 win
blue team :  브라질리언했음 win
blue team :  머스켓 win
blue team :  피들장인 최빨대 win
red team :  탐스런콧구멍 loose
red team :  까치래도차케 loose
red team :  horosha loose
red team :  시로조아 loose
red team :  Ezreal Lover loose


In [13]:
match_hist[0].participants[summoner.name].champion.id

6

In [14]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter


def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

    # match_history = cass.get_match_history(summoner=summoner, seasons={Season.season_8}, queues={Queue.ranked_solo_fives})
    match_history = cass.get_match_history(summoner)
    match_history(seasons={Season.season_8}, queues={Queue.ranked_solo_fives})

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {champion.id: champion.name for champion in cass.get_champions(region=region)}
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner.name].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print("Top 10 champions {} played:".format(summoner.name))
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

print_newest_match(name='벤젠덴겐', region='KR')


Making call: https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/209691368?beginIndex=0&endIndex=100
Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/item.json
Making call: https://ddragon.leagueoflegends.com/cdn/8.24.1/data/ko_KR/runesReforged.json


ValueError: '영감' is not a valid RunePath

In [ ]:
import cassiopeia as cass
from cassiopeia import Summoner, Match
from cassiopeia.data import Season, Queue
from collections import Counter


def print_newest_match(name: str, region: str):

    # Notice how this function never makes a call to the summoner endpoint because we provide all the needed data!

    summoner = Summoner(name=name, region=region)

    # A MatchHistory is a lazy list, meaning it's elements only get loaded as-needed.

    # match_history = cass.get_match_history(summoner=summoner, seasons={Season.season_8}, queues={Queue.ranked_solo_fives})
    match_history = summoner.match_history
    match_history(seasons={Season.season_8}, queues={Queue.ranked_solo_fives})

    # Load the entire match history by iterating over all its elements so that we know how long it is.
    # Unfortunately since we are iterating over the match history and accessing the summoner's champion for each match,
    # we need to know what version of the static data the champion should have. To avoid pulling many different
    # static data versions, we will instead create a {champion_id -> champion_name} mapping and just access the champion's
    # ID from the match data (which it provides directly).
    champion_id_to_name_mapping = {champion.id: champion.name for champion in cass.get_champions(region=region)}
    played_champions = Counter()
    for match in match_history:
        champion_id = match.participants[summoner.name].champion.id
        champion_name = champion_id_to_name_mapping[champion_id]
        played_champions[champion_name] += 1
    print("Length of match history:", len(match_history))

    # Print the aggregated champion results
    print("Top 10 champions {} played:".format(summoner.name))
    for champion_name, count in played_champions.most_common(10):
        print(champion_name, count)
    print()

    match = match_history[0]
    print('Match ID:', match.id)

    p = match.participants[summoner]
    print("\nSince the match was created from a matchref, we only know one participant:")
    #print(p.summoner.name, 'playing', p.champion.name)
    print(p.id, p.summoner.region,  p.summoner.name, p.summoner.id, p.champion.id)

    print("\nNow pull the full match data by iterating over all the participants:")
    for p in match.participants:
        #print(p.summoner.name, 'playing', p.champion.name)
        print(p.id, p.summoner.region, p.summoner.name, p.summoner.id, p.champion.id, p.team.first_dragon)

    print("\nIterate over all the participants again and note that the data is not repulled:")
    for p in match.participants:
        #print(p.summoner.name, 'playing', p.champion.name)
        print(p.id, p.summoner.region,  p.summoner.name, p.summoner.id, p.champion.id, p.team.first_dragon)

    print("\nBlue team won?", match.blue_team.win)
    print("Red team won?", match.red_team.win)
    print("Participants on blue team:")
    for p in match.blue_team.participants:
        print(p.summoner.name)


if __name__ == "__main__":
    print_newest_match(name="머스켓", region="KR")

In [15]:
cass.get_match(id=3475505515).participants[0].summoner.name

'벤젠덴겐'

In [22]:
cass.get_match(id=3475505519).region

<Region.korea: 'KR'>

In [17]:
# match_id 3473000000~3475000000

In [18]:
cass.get_match(id=3473500000, region='KR').teams#.season

Making call: https://kr.api.riotgames.com/lol/match/v3/matches/3473500000


NotFoundError: No source returned a query result!